In [2]:
import pandas as pd
%matplotlib inline
%run Pipeline//upload_and_vizualize 
%run Pipeline//classify_and_evaluate 
%run Pipeline//aux
%run Pipeline//ULAB_ML_Pipeline
%run processing
import numpy as np 
import matplotlib.pyplot as plt
import pylab
import sys
import random
import sklearn as sk 
import json 
import re
from multiprocessing import Pool
from functools import partial
from time import time
from sklearn import svm, ensemble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit, RandomizedLogisticRegression
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cross_validation import train_test_split, KFold
from sklearn.preprocessing import *
from sklearn.feature_selection import RFE
from sklearn.grid_search import ParameterGrid
from sklearn.svm import LinearSVC
from sklearn.metrics import *
import csv
from errno import EEXIST
from os import makedirs,path
from datetime import datetime as dr
from datetime import date
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')
import pylab as pl
from upload_and_vizualize import camel_to_snake
from datetime import datetime as dt
from datetime import date

In [170]:
### SHOULD BE WITHIN CLEANING.PY ###

## FROM PREPROCESSING ##

#interest_var = ['PGM_SYS_ID','ACTIVITY_ID','AGENCY_TYPE_DESC','STATE_CODE','AIR_LCON_CODE','COMP_DETERMINATION_UID','ENF_RESPONSE_POLICY_CODE','PROGRAM_CODES']
def replace_with_value(data_file, variables, values):
    '''
    '''
    for variable in variables:
        value = values[variables.index(variable)]
        data_file[variable] = data_file[variable].fillna(value)

def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)

def to_date_time(df, date_format, date_col):
    #add datetime column
    df[date_col+'_datetime'] = df[date_col].apply(convert_to_datetime, date_format=date_format)
    df[date_col+'_year'] = df[date_col+'_datetime'].apply(convert_to_year)

    #return df

def convert_to_month(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.month)

def get_month_year_col(df, date_column, date_format):
    df[date_column+'_datetime'] = df[date_column].apply(convert_to_datetime, date_format=date_format)
    df[date_column+'_month'] = df[date_column+'_datetime'].apply(convert_to_month)
    df[date_column+'_year'] = df[date_column+'_datetime'].apply(convert_to_year)
    return df

def filter_date(df, date_format, date_col, start=None, end=None):
    to_date_time(df, date_format, date_col)
    
    
    if start:
        timestart = dt.strptime(start,"%Y/%m/%d")
        #print(start)
        df = df[df[date_col+'_datetime'] >= timestart ]
        #print(df.head())
    if end:
        timeend = dt.strptime(end,"%Y/%m/%d")
        #print(end)
        df = df[df[date_col+'_datetime'] <= timeend ]
        #print(df.head())
    
    return df

def filter_col(df, fac_id, features, date_col):
    #filter needed
    df = df[[fac_id] + [date_col+'_datetime'] + [date_col+'_year'] + features]
    return df




## FROM PREPROCESSING, MODIFIED ##
def add_dummy(df, variable_list, sep_char = None, drop_one=False, drop_original=False):
    '''
    Input: 
        - df: pandas dataframe
        - variable_list: a list of variables to dummitize
        - drop_one: whether to drop first dummy
        - drop_original: whether to drop original categorical variable
    Output: dataframe with tht dummy variables added
    '''
    for variable in variable_list:
        if sep_char:
            df_dummy = df[variable].str.get_dummies(sep=sep_char)
            df_dummy.columns = [variable+ '_' +str(col) for col in df_dummy.columns]

        else:
            df_dummy = pd.get_dummies(df[variable], drop_first=drop_one, prefix = variable)
        
        df = pd.concat([df, df_dummy], axis=1)
        if drop_original:
            df = df.drop(variable, 1)
    return (df, df_dummy.columns)

def aggr_dummy_cols(df, final_df, colnames, mode = None):
    for col in colnames:
        
        cross = pd.crosstab(df['id_+_date'], columns=df[col])
        
        if mode == 'cat':
            cross.columns = [cross.columns.name+ '_' +str(col) for col in cross.columns]
        
        elif mode == 'dum':
            cross = cross.drop(0, axis = 1)
            cross.columns = [cross.columns.name for col in cross.columns]
            
        #SIMPLIFY THIS!!
        elif mode == 'bim':
            cross = cross.drop('N', axis = 1)
            cross.columns = [cross.columns.name for col in cross.columns]
        
        #ADD PROCESSING CONT_VAR
        
        else:
            cross.columns = [cross.columns.name for col in cross.columns]
            
        cross.columns.name = None
        cross.reset_index(inplace=True)
        
        
        if final_df.empty:
            final_df = final_df.append(cross)
        else:
            final_df = pd.merge(final_df, cross, how = 'left', on = 'id_+_date')
            
    return final_df

## FROM ULAB PIPELINE ##

def generate_continous_variable(data_file, variable_list):
    '''
    function that can take a categorical variable and create 
    binary variables from it
    '''
    for variable in variable_list:
        list_values = list(data_file.groupby(variable).groups.keys())
        for i,value in enumerate(list_values):
            data_file[variable].replace(value,i)

    return data_file 



In [156]:
FACILITY = 'data/ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv'
VIOLATION = 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv'
INSPECTION = 'data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv'
FORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv'
INFORMALACT = 'data/ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv'
STACKTEST = 'data/ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv'
TITLEV = 'data/ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv'

#facilities = read_file('ICIS-AIR_downloads/ICIS-AIR_FACILITIES.csv')
#pd_violations = read_file('ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
#pd_inspection = read_file('ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
#pd_formalact = read_file('ICIS-AIR_downloads/ICIS-AIR_FORMAL_ACTIONS.csv')
#pd_informalact = read_file('ICIS-AIR_downloads/ICIS-AIR_INFORMAL_ACTIONS.csv')
#pd_stacktest = read_file('ICIS-AIR_downloads/ICIS-AIR_STACK_TESTS.csv')
#pd_titlev = read_file('ICIS-AIR_downloads/ICIS-AIR_TITLEV_CERTS.csv')

In [157]:
## CONFIG DATA ##
START_DATE= '2007/01/01'
END_DATE = '2016/12/31'
fac_id = 'PGM_SYS_ID'
df_dict ={'violation': {#'data_file': 'data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv',
                        'interest_var': ['AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_RESOLVED_DATE'],
                         'date_col': 'HPV_DAYZERO_DATE',
                       'date_format':'%m-%d-%Y'},
          
           'inspection': {'interest_var': ['STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES'],
                         'date_col': 'ACTUAL_END_DATE',
                         'date_format':'%m-%d-%Y'},
          
           'stacktest': {'interest_var':['COMP_MONITOR_TYPE_CODE','POLLUTANT_CODES','AIR_STACK_TEST_STATUS_CODE'],
                        'date_col': 'ACTUAL_END_DATE',
                        'date_format':'%m/%d/%Y'},
          
           'titlev':{'interest_var':['COMP_MONITOR_TYPE_CODE','FACILITY_RPT_DEVIATION_FLAG'],
                        'date_col': 'ACTUAL_END_DATE',
                    'date_format':'%m/%d/%Y'},
          
           'formalact':{'interest_var':['ENF_TYPE_CODE','PENALTY_AMOUNT'],
                        'date_col': 'SETTLEMENT_ENTERED_DATE',
                       'date_format':'%m/%d/%Y'},
          
           'informalact':{'interest_var':['ENF_TYPE_CODE'],
                        'date_col': 'ACHIEVED_DATE',
                         'date_format':'%m/%d/%Y'}}

#general_df = pd.DataFrame()

In [173]:
## READ INITIAL FILTERED FILE ##
violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)

In [210]:
violation = read_file('data/ICIS-AIR_downloads/ICIS-AIR_VIOLATION_HISTORY.csv')
inspection = read_file('data/ICIS-AIR_downloads/ICIS-AIR_FCES_PCES.csv')
date_types = ['year']
date_format ='%m-%d-%Y'
date_col = ['HPV_DAYZERO_DATE','HPV_RESOLVED_DATE','EARLIEST_FRV_DETERM_DATE']
get_occupied_frame(violation,date_col,date_format,date_types)

date_col = ['ACTUAL_END_DATE']
get_occupied_frame(inspection,date_col,date_format,date_types)

fce = inspection[['PGM_SYS_ID','STATE_EPA_FLAG','COMP_MONITOR_TYPE_CODE','PROGRAM_CODES','ACTUAL_END_DATE','ACTUAL_END_DATE_year','ACTUAL_END_DATE_datetime']]
violation = violation[['PGM_SYS_ID','AGENCY_TYPE_DESC','AIR_LCON_CODE','ENF_RESPONSE_POLICY_CODE','POLLUTANT_CODES','PROGRAM_CODES','HPV_DAYZERO_DATE','HPV_DAYZERO_DATE_year','HPV_DAYZERO_DATE_datetime']]
violation = violation[violation.ENF_RESPONSE_POLICY_CODE == 'HPV']

In [45]:
def generate_features(start_date, end_date):
    #violation, inspection, titlev, stacktest, formalact, informalact = general_read_file(df_dict, START_DATE, END_DATE)
    
    violation_df = process_violation(violation, start_date, end_date)
    inspection_df = process_titlev(titlev, start_date, end_date)
    stacktest_df = process_stacktest(stacktest, start_date, end_date)
    formalact_df = process_formalact(formalact, start_date, end_date)
    informalact_df = process_informalact(informalact, start_date, end_date)
    noninspectHPV_df = process_noninspectHPV(violation, inspection, start_date, end_date)
    
    final_df = pd.merge(inspection_df, violation_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, stacktest_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, formalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, informalact_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])
    final_df = pd.merge(final_df, noninspectHPV_df, how = 'left', right_on = ["id_+_date"], left_on = ["id_+_date"])

    re_separate = r'(.[^_]*)_(.*)'
    sep = lambda x: pd.Series([i for i in re.split(re_separate,x)])
    final_id_year = final_df['id_+_date'].apply(sep)
    final_df =pd.concat([final_id_year.rename(columns={1:'PGM_SYS_ID',2:'HPV_DAYZERO_DATE_year'}), final_df], axis=1)
    final_df.drop([0, 3, 'id_+_date'], axis = 1, inplace = True)

    return final_df

In [46]:
#Checking for feature generation
start_date = '2009'
end_date = '2012'
feat = generate_features(start_date, end_date)
feat.shape

(56246, 173)

In [175]:
violation.head()

,PGM_SYS_ID,HPV_DAYZERO_DATE_datetime,HPV_DAYZERO_DATE_year,AGENCY_TYPE_DESC,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_RESOLVED_DATE
5,CT0000000900508907,2010-03-17,2010,State,NaN,HPV,300000329,CAAFESOP,11-02-2010
6,CT0000000900100078,2007-08-08,2007,State,NaN,HPV,300000329,CAAFESOP,09-24-2009
7,CT0000000901501125,2013-04-26,2013,State,NaN,HPV,10193,CAATVP,06-25-2014
9,CT0000000900109058,2008-01-17,2008,State,NaN,HPV,300000005,CAAFESOP,01-06-2009
13,CT0000000901100054,2008-04-08,2008,State,NaN,HPV,10461 300000005,CAATVP,12-10-2008


In [12]:
#### FAILED LABEL #####

In [211]:
def generate_label(violhist, fce, start_year, end_year):
    #removing FRVs
    #violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE != 'FRV']
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] >= start_year]
    violhist = violhist[violhist['HPV_DAYZERO_DATE_year'] <= end_year]
    
    #for fce
    fce = fce[fce['ACTUAL_END_DATE_year'] >= start_year]
    fce = fce[fce['ACTUAL_END_DATE_year'] <= end_year]
    
    merged_hpv_fce = pd.merge(violhist, fce, how='right', left_on=['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], right_on=['PGM_SYS_ID','ACTUAL_END_DATE'])
    
    #finding 0's
    '''
    non_viol = merged_hpv_fce[merged_hpv_fce.HPV_DAYZERO_DATE.isnull()]
    non_viol['is_violation'] = 0
    '''
    #non_viol = merged_hpv_fce
    #non_viol.HPV_DAYZERO_DATE.fillna(0, inplace=True)
    #non_viol = non_viol.filter(['PGM_SYS_ID', 'ACTUAL_END_DATE', 'is_violation', 'ACTUAL_END_DATE_year'], axis = 1)
    #***non_viol = non_viol[['PGM_SYS_ID', 'ACTUAL_END_DATE', 'ACTUAL_END_DATE_year']]
    #non_viol.rename(columns={'ACTUAL_END_DATE': 'HPV_DAYZERO_DATE', 'ACTUAL_END_DATE_year': "HPV_DAYZERO_DATE_year" }, inplace=True)
    #finding 1's
    #viol = merged_hpv_fce[merged_hpv_fce.HPV_DAYZERO_DATE.notnull()]
    #***viol = merged_hpv_fce
    #***viol.HPV_DAYZERO_DATE.fillna(0, inplace=True)
    #viol['is_violation'] = 1
    #viol = viol.filter(['PGM_SYS_ID', 'HPV_DAYZERO_DATE', 'is_violation', 'HPV_DAYZERO_DATE_year'], axis = 1)
    #***viol = viol[['PGM_SYS_ID', 'HPV_DAYZERO_DATE', 'HPV_DAYZERO_DATE_year']]
    #***output = pd.concat([viol,non_viol])
    #output['id_+_date'] = output.PGM_SYS_ID +'_'+ output.HPV_DAYZERO_DATE_year
    #output.drop(['PGM_SYS_ID', 'HPV_DAYZERO_DATE', 'HPV_DAYZERO_DATE_year'], axis = 1, inplace = True)
    #output = output.groupby('id_+_date').size().reset_index()
    #output.rename(columns = {0:"outcome"}, inplace = True)
    #return output
    return merged_hpv_fce, violhist, fce

In [282]:
out, violhist, fce = generate_label(violation, inspection, start_date, end_date)

In [213]:
violation.head()

,PGM_SYS_ID,AGENCY_TYPE_DESC,AIR_LCON_CODE,ENF_RESPONSE_POLICY_CODE,POLLUTANT_CODES,PROGRAM_CODES,HPV_DAYZERO_DATE,HPV_DAYZERO_DATE_year,HPV_DAYZERO_DATE_datetime
0,CT0000000900700108,State,NaN,HPV,300000005 300000323,CAASIP,05-28-1996,1996,1996-05-28
2,CT0000000900900110,State,NaN,HPV,300000005 300000323,CAASIP,05-28-1996,1996,1996-05-28
3,CT0000000900900110,State,NaN,HPV,10461 300000005,CAASIP,08-21-1998,1998,1998-08-21
4,CT0000000900300125,State,NaN,HPV,300000242 300000329,CAASIP,07-12-2005,2005,2005-07-12
5,CT0000000900508907,State,NaN,HPV,300000329,CAAFESOP,03-17-2010,2010,2010-03-17


In [306]:
out.HPV_DAYZERO_DATE.fillna(0, inplace=True)

def change_to_zero(series_row):
    if type(series_row) != str:
        return 0
    else:
        return 1

out['Outcome'] = out.HPV_DAYZERO_DATE.apply(change_to_zero)
#test = out.HPV_DAYZERO_DATE_year.replace(~0, 1)

In [297]:
out.Outcome.value_counts()

0    290159
1      1839
Name: Outcome, dtype: int64

In [300]:
output_ay = out.groupby(['PGM_SYS_ID', 'ACTUAL_END_DATE_year']).sum().reset_index()

In [308]:
def change_to_zero_float(series_row):
    if series_row > 1:
        return 1
    else:
        return 0
    
output_ay.Outcome = output_ay.Outcome.apply(change_to_zero_float)

In [310]:
output_ay.ACTUAL_END_DATE_year.value_counts()

2009    37380
2011    36210
2012    36007
2010    35867
Name: ACTUAL_END_DATE_year, dtype: int64